In [257]:
import requests

ip = '62.68.146.67'
#ip = 'localhost'
url = f"http://{ip}:8000/reset_database"

response = requests.post(url)
response.json()

{'status': 'success', 'message': 'База данных сброшена, все коллекции очищены'}

In [258]:
import requests

url = f"http://{ip}:8000/add_from_sheet"

sheet_data = {
    "sheet_id": "1qEormopG5VJDg9BfG4-d5kRBTdMPksRcN2sXar9Xepo",
    "gid": "0"
}

response = requests.post(url, json=sheet_data)
response.json()

{'status': 'success',
 'message': 'Added 1659 new unique question-answer pairs',
 'new_pairs_count': 1659,
 'duplicates_skipped': 0,
 'total_rows_in_sheet': 1659,
 'used_question_column': 'question',
 'used_answer_column': 'content'}

In [259]:
import requests

url = f"http://{ip}:8000/view_top_n"

params = {
    "n": 5
}

response = requests.get(url, params=params)
response.json()

{'total_items': 5,
 'items': [{'id': '0',
   'question': 'Какие документы необходимо предоставить при приеме беженцев?',
   'answer': 'Вариант 1: 1) действующее удостоверение беженца: документ, удостоверяющий личность на территории РФ; 2) регистрация по месту пребывания; 3) СНИЛС (при наличии); 4) трудовая книжка и/или сведения о трудовой деятельности (при наличии трудового стажа). Вариант 2: 1) действующее свидетельство о предоставлении временного убежища: документ, удостоверяющий личность на территории РФ; 2) может быть предоставлен паспорт, но основанием для трудоустройства будет именно свидетельство; 3) регистрация по месту пребывания; 4) СНИЛС (при наличии); 5) трудовая книжка и/или сведения о трудовой деятельности (при наличии трудового стажа). Вариант 3: 1) паспорт гражданина Украины, Донецкой Народной Республики или Луганской Народной Республики; 2) документ, подтверждающий прохождение дактилоскопии и фотографирования. Данные должны совпадать с данными в паспорте; 3) регистраци

In [260]:
import requests

url = f"http://{ip}:8000/count_items"

response = requests.get(url)
print(response.json())

{'collection_name': 'default', 'item_count': 1659}


In [261]:
import requests

url = f"http://{ip}:8000/api/v1/get_answer/"

data = {
    "queries": ["как провести отпуск"],
    "n_results": 3
}

response = requests.post(url, json=data)
response.json()

{'results': [{'query': 'как провести отпуск',
   'results': [{'id': '113',
     'question': 'Как провести отпуск?',
     'answer': 'Оформление отпуска по заявлению возможно двумя способами: 1. в разделе "Отпуска" в левом меню Личного кабинета сотрудника (https://company-x5.ru/vacations/plan?vp_page=1 ); 2. в разделе "Заявки", группа "Отпуск", плитка "Отпуск по заявлению (ЛК)". (https://company-x5.ru/requests/tiles/my/)',
     'distance': 0.17390930652618408},
    {'id': '448',
     'question': 'как оформить отпуск',
     'answer': 'Создать заявку на отпуск можно в разделе "Сервис", подразделе "Отпуск". Требования к запрашиваемому отпуску вы сможете видеть левой части экрана',
     'distance': 0.2272828221321106},
    {'id': '447',
     'question': 'как создать заявку на отпуск?',
     'answer': 'Создать заявку на отпуск можно в разделе "Сервис", подразделе "Отпуск". Требования к запрашиваемому отпуску вы сможете видеть левой части экрана',
     'distance': 0.2599191665649414}]}]}

In [155]:
import requests
import json

url = "http://62.68.146.67:11434/api/chat/"

# Пример использования
prompt = "Привет"

payload = {
    "model": "gemma2:2b-instruct-q8_0",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "stream": False
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, headers=headers, data=json.dumps(payload))
response.json()

{'model': 'gemma2:2b-instruct-q8_0',
 'created_at': '2024-09-12T15:10:22.939073833Z',
 'message': {'role': 'assistant',
  'content': 'Привет! 👋  😊\n\nЧем я могу тебе помочь? \n'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 445367920,
 'load_duration': 38965326,
 'prompt_eval_count': 10,
 'prompt_eval_duration': 35270000,
 'eval_count': 15,
 'eval_duration': 369299000}

In [266]:
import requests
import json
from typing import List, Dict, Any
from collections import defaultdict

def rag_query(ip: str, queries: List[str], n_results: int) -> Dict[str, Any]:
    """
    Perform a RAG query to retrieve relevant information.
    """
    url = f"http://{ip}:8000/api/v1/get_answer/"
    data = {
        "queries": queries,
        "n_results": n_results
    }
    response = requests.post(url, json=data)
    result = response.json()
    #print("RAG Response:", json.dumps(result, indent=2, ensure_ascii=False))
    return result

def llm_query(prompt: str, model: str = "gemma2:2b-instruct-q8_0") -> Dict[str, Any]:
    """
    Perform an LLM query using the given prompt.
    """
    url = f"http://{ip}:11434/api/chat/"
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False
    }
    headers = {
        "Content-Type": "application/json"
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    return response.json()


def create_llm_prompt(user_question: str, rag_response: Dict[str, Any]) -> str:
    """
    Create a prompt for the LLM based on the user question and RAG results,
    with question grouping for identical answers.
    """
    prompt = f'Вопрос от пользователя: "{user_question}"\n\nРелевантные вопросы и ответы из базы знаний:\n'
    
    # Extract results from the RAG response
    rag_results = rag_response['results'][0]['results']
    
    # Group questions by their answers
    answer_to_questions = defaultdict(list)
    for result in rag_results:
        answer_to_questions[result['answer']].append(result['question'])
    
    # Create condensed Q&A pairs
    for i, (answer, questions) in enumerate(answer_to_questions.items(), 1):
        if len(questions) > 1:
            formatted_questions = ', '.join(f'"{q}"' for q in questions)
            prompt += f"#{i}. Для вопросов: {formatted_questions}\n   Ответ: {answer}\n\n"
        else:
            prompt += f"#{i}. \"{questions[0]}\"\n   Ответ: {answer}\n\n"
    
    prompt += '''
Задача: Основываясь на вспомогательных материалах корпоративной базы знаний компании x5 retail group, развёрнуто и чётко ответить на вопрос пользователя. 
Важно, что вспомогательные данные не всегда ранжированы по релевантности. 
Среди примеров может не быть верного ответа! 
Если ты считаешь, вопрос пользователя релевантен вопросу из базы знаний, дай соответствующий ответ. 
Можно перефразировать его немного для разнообразия, но не в ущерб смыслу! 
Тебе необходимо ответить на вопрос. Объяснять, почему ты выбрал именно это вариант из базы не нужно. 
Если ты считаешь, что в предоставленных данных не содержится ответа на вопрос, то нужно ответить, что тебе дали слишком мало конкретики. Попроси описать проблему подробнее. 
Если в вспомогательных данных есть несколько вопросов, которые релевантны данному, задай уточняющий вопрос, чтобы понять, какой из вопросов больше подходит.
При ответе на вопрос ничего лишнего не пиши. Только сам ответ или уточняющее предложение.
Ответ должен быть полным, как в базе знаний. Допускаются лишь незначительные отвклонения. Из базы знаний нужно выбрать лишь один ответ!
Отвечай только на русском языке.'''
    
    return prompt

def rag_llm_pipeline(ip: str, user_question: str, n_results: int = 5) -> Dict[str, Any]:
    """
    Perform the full RAG-LLM pipeline for a given user question.
    """
    # 1. Perform RAG query
    rag_response = rag_query(ip, [user_question], n_results)
    
    # 2. Create LLM prompt
    llm_prompt = create_llm_prompt(user_question, rag_response)
    
    # 3. Perform LLM query
    llm_response = llm_query(llm_prompt)
    
    return {
        "user_question": user_question,
        "rag_results": rag_response,
        "llm_prompt": llm_prompt,
        "llm_response": llm_response
    }

In [277]:
answer = rag_llm_pipeline(ip, "у меня зависла задача!")

In [278]:
answer['llm_response']

{'model': 'gemma2:2b-instruct-q8_0',
 'created_at': '2024-09-12T16:14:14.262655667Z',
 'message': {'role': 'assistant',
  'content': 'Создайте, пожалуйста, обращение в ИТ поддержку на портале support \n'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 593868968,
 'load_duration': 43207425,
 'prompt_eval_count': 378,
 'prompt_eval_duration': 209776000,
 'eval_count': 18,
 'eval_duration': 338986000}

In [276]:
answer

{'user_question': 'как мне компенсировать родственникам убитого сотрудника его смерть',
 'rag_results': {'results': [{'query': 'как мне компенсировать родственникам убитого сотрудника его смерть',
    'results': [{'id': '325',
      'question': 'документы на материальную помощь при смерти родственника',
      'answer': 'Заявление на оказание материальной помощи; Справка или свидетельство о смерти; Документы, подтверждающие родство.',
      'distance': 0.2937978506088257},
     {'id': '422',
      'question': 'как получить выплату по уходу за больным родственником от работодателя?',
      'answer': 'Больничный с кодом 09 (уход за больным членом семьи) оплачивается полностью за счет средств СФР, работодатель в данном случае выплату не производит.',
      'distance': 0.4016841650009155},
     {'id': '39',
      'question': 'Сотрудник скончался его уже нет в команде магазина , я должна оформить материальную помощь его родственникам .как мне это сделать если его уже нет в компании , а заявк